In [ ]:
from scipy.io import loadmat
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
from IPython.display import display
import warnings
warnings.filterwarnings("ignore")
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN, LSTM
from keras.layers import Dropout
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error,mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler

In [ ]:
train_data_file = pd.read_csv("train_data2.csv")
print(train_data_file.columns)

In [ ]:
train_price = train_data_file['"HDFCBANK_M&M"']
plt.plot(train_price)

In [ ]:
train_data_file['"HDFCBANK_M&M"']

In [ ]:
test_data_file = pd.read_csv("test_data2.csv")
print(test_data_file.columns)

In [ ]:
test_price = test_data_file['"RELIANCE_UPL"']
plt.figure(figsize = (30,10))
plt.plot(test_price)

In [ ]:
names = train_data_file.columns
names = list(names)
print(names)

In [ ]:
data = {"A":[1,2,3],"b":[3,4,5],"c":[4,5,6]}
df = pd.DataFrame(data, columns = ["A","b","c"])

In [ ]:
df.to_csv("sample_out.csv")

In [ ]:
df["A"]

In [ ]:
predicted_series_rnn = {}
predicted_series_lstm = {}

In [ ]:
rows = []
count = 0
for file in names:
    print("\n***************************************************************************************************************")
    print("Pair Number:", count)
    count+=1
    train_data = train_data_file[file]
    test_data = test_data_file[file]
    length_validation = len(test_data)
    length_train = len(train_data)

    train = np.array(train_data)
    dataset_train = train
    dataset_train = train.reshape(-1,1)
    
    scaler = MinMaxScaler(feature_range = (0,1))
    
    # scaling dataset
    dataset_train_scaled = scaler.fit_transform(dataset_train)
    
    X_train = []
    y_train = []
    
    time_step = 50
    
    for i in range(time_step, length_train):
        X_train.append(dataset_train_scaled[i-time_step:i,0])
        y_train.append(dataset_train_scaled[i,0])
        
    # convert list to array
    X_train, y_train = np.array(X_train), np.array(y_train)
    
    
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
    y_train = np.reshape(y_train, (y_train.shape[0],1))
    
    
    # initializing the RNN
    regressor = Sequential()
    
    # adding first RNN layer and dropout regulatization
    regressor.add(SimpleRNN(units = 50, activation = "tanh", return_sequences = True, input_shape = (X_train.shape[1],1)))
    
    regressor.add(Dropout(0.2))
    
    # # adding third RNN layer and dropout regulatization
    
    regressor.add(SimpleRNN(units = 50, activation = "tanh", return_sequences = True))
    
    regressor.add(Dropout(0.2))
    
    # adding fourth RNN layer and dropout regulatization
    
    regressor.add(SimpleRNN(units = 50))
    
    regressor.add(Dropout(0.2))
    
    # adding the output layer
    regressor.add(Dense(units = 1))
    
    # # compiling RNN
    regressor.compile(
        optimizer = "adam", 
        loss = "mean_squared_error")
    
    # fitting the RNN
    history = regressor.fit(X_train, y_train, epochs = 35, batch_size = 32, shuffle = False)

    y_train = scaler.inverse_transform(y_train) # scaling back from 0-1 to original
    y_pred = regressor.predict(X_train)  # predictions
    y_pred = scaler.inverse_transform(y_pred) # scaling back from 0-1 to original

    
    validation_data = np.array(test_data)
    dataset_validation = validation_data  # getting "open" column and converting to array
    dataset_validation = np.reshape(dataset_validation, (-1,1))  # converting 1D to 2D array
    scaled_dataset_validation =  scaler.fit_transform(dataset_validation)  # scaling open values to between 0 and 1
    
    # Creating X_test and y_test
    X_test = []
    y_test = []
    
    for i in range(time_step, length_validation):
        X_test.append(scaled_dataset_validation[i-time_step:i,0])
        y_test.append(scaled_dataset_validation[i,0])
    
    # Converting to array
    X_test, y_test = np.array(X_test), np.array(y_test)
    
    X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))  # reshape to 3D array
    y_test = np.reshape(y_test, (-1,1))  # reshape to 2D array
     
    # predictions with X_test data
    y_pred_of_test = regressor.predict(X_test)
    # scaling back from 0-1 to original
    y_pred_of_test = scaler.inverse_transform(y_pred_of_test) 
    flattened_results = np.array(y_pred_of_test).flatten().tolist()
    predicted_series_rnn[names.index(file)] = flattened_results
    # visualisation
    plt.figure(figsize = (30,10))
    plt.plot(y_pred_of_test, label = "y_pred_of_test", c = "orange")
    plt.plot(scaler.inverse_transform(y_test) , label = "y_test", c = "g")
    plt.xlabel("time")
    plt.ylabel("distance")
    plt.title("Testing Graph")
    plt.legend()
    plt.show()
    
    R2score = r2_score(scaler.inverse_transform(y_test), y_pred_of_test)
    rsme = np.sqrt(mean_squared_error(scaler.inverse_transform(y_test), y_pred_of_test))
    mape = mean_absolute_percentage_error(scaler.inverse_transform(y_test), y_pred_of_test)
    mae = mean_absolute_error(scaler.inverse_transform(y_test), y_pred_of_test)
    print("\nPrediction results:")
    results_rnn = [R2score, rsme, mae, mape]
    y = scaler.inverse_transform(y_test)
    # thresholds = [epsilon1[names.index(file)],epsilon2[names.index(file)],epsilon3[names.index(file)],epsilon4[names.index(file)]]
    
    print("r2_score: ", R2score)
    print("MAPE: ", mape)
    print("MAE: ", mae)
    print("RMSE: ", rsme)
   
    print("_____________________________")
    
    
    ###LSTM#####
    
    train_data = train_data_file[file]
    test_data = test_data_file[file]
    length_validation = len(test_data)
    length_train = len(train_data)
    
    train = np.array(train_data)
    dataset_train = train
    dataset_train = train.reshape(-1,1)
    
    scaler = MinMaxScaler(feature_range = (0,1))
    
    # scaling dataset
    dataset_train_scaled = scaler.fit_transform(dataset_train)
    
    X_train = []
    y_train = []
    
    time_step = 50
    
    for i in range(time_step, length_train):
        X_train.append(dataset_train_scaled[i-time_step:i,0])
        y_train.append(dataset_train_scaled[i,0])
        
    # convert list to array
    X_train, y_train = np.array(X_train), np.array(y_train)
    
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
    y_train = np.reshape(y_train, (y_train.shape[0],1))
    
    #Initialise LSTM
    regressor = Sequential()
    
    regressor.add(LSTM(units = 50, activation = "tanh", return_sequences = True, input_shape = (X_train.shape[1],1)))
    regressor.add(Dropout(0.2))
    
    regressor.add(LSTM(units = 50, activation = "tanh"))
    regressor.add(Dropout(0.2))
    
    regressor.add(Dense(units = 1))
    
    # # compiling RNN
    regressor.compile(
        optimizer = "adam", 
        loss = "mean_squared_error")
    
    # fitting the RNN
    history = regressor.fit(X_train, y_train, epochs = 35, batch_size = 32, shuffle = False)
    
    y_train = scaler.inverse_transform(y_train) # scaling back from 0-1 to original
    y_pred = regressor.predict(X_train)  # predictions
    y_pred = scaler.inverse_transform(y_pred) # scaling back from 0-1 to original
    
    validation_data = np.array(test_data)
    dataset_validation = validation_data  # getting "open" column and converting to array
    dataset_validation = np.reshape(dataset_validation, (-1,1))  # converting 1D to 2D array
    scaled_dataset_validation =  scaler.fit_transform(dataset_validation)  # scaling open values to between 0 and 1
    
    # Creating X_test and y_test
    X_test = []
    y_test = []
    
    for i in range(time_step, length_validation):
        X_test.append(scaled_dataset_validation[i-time_step:i,0])
        y_test.append(scaled_dataset_validation[i,0])
    
    # Converting to array
    X_test, y_test = np.array(X_test), np.array(y_test)

    
    X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))  # reshape to 3D array
    y_test = np.reshape(y_test, (-1,1))  # reshape to 2D array
    
    
    # predictions with X_test data
    y_pred_of_test = regressor.predict(X_test)
    # scaling back from 0-1 to original
    y_pred_of_test = scaler.inverse_transform(y_pred_of_test) 
    flattened_results = np.array(y_pred_of_test).flatten().tolist()
    predicted_series_lstm[names.index(file)] = flattened_results
    # visualisation
    plt.figure(figsize = (30,10))
    plt.plot(y_pred_of_test, label = "y_pred_of_test", c = "orange")
    plt.plot(scaler.inverse_transform(y_test), label = "y_test", c = "green")
    plt.xlabel("time")
    plt.ylabel("distance")
    plt.title("Testing Graph")
    plt.legend()
    plt.show()
    
    R2score = r2_score(scaler.inverse_transform(y_test), y_pred_of_test)
    rsme = np.sqrt(mean_squared_error(scaler.inverse_transform(y_test), y_pred_of_test))
    mape = mean_absolute_percentage_error(scaler.inverse_transform(y_test), y_pred_of_test)
    mae = mean_absolute_error(scaler.inverse_transform(y_test), y_pred_of_test)
    print("\nPrediction results:")
    
    results_lstm = [R2score, rsme, mae, mape]
    y = scaler.inverse_transform(y_test)
    # thresholds = [epsilon1[names.index(file)],epsilon2[names.index(file)],epsilon3[names.index(file)],epsilon4[names.index(file)]]
    
    
    print("r2_score: ", R2score)
    print("MAPE: ", mape)
    print("MAE: ", mae)
    print("RSME: ", rsme)

    final_results = [file]
    final_results.append(results_rnn)
    final_results.append(results_lstm)
    print(final_results)
    rows.append(final_results)

In [ ]:
df = pd.DataFrame(predicted_series_rnn)
df.columns = names
df.to_csv("predicted_series_rnn_block10_w50.csv")

In [ ]:
df = pd.DataFrame(predicted_series_lstm)
df.columns = names
df.to_csv("predicted_series_lstm_block10_w50.csv")

In [ ]:
rnn_r2 = []
rnn_rmse = []
rnn_mae = []
rnn_mape = []
lstm_r2 = []
lstm_rmse = []
lstm_mae = []
lstm_mape = []

results_rows = []
for i in range (len(rows)):
    row = rows[i]
    rnn = row[1]
    rnn_r2.append(rnn[0])
    rnn_rmse.append(rnn[1])
    rnn_mae.append(rnn[2])
    rnn_mape.append(rnn[3])
    
    lstm = row[2]
    
    lstm_r2.append(lstm[0])
    lstm_rmse.append(lstm[1])
    lstm_mae.append(lstm[2])
    lstm_mape.append(lstm[3])
    
results_rows = [rnn_r2,rnn_rmse,rnn_mae,
                 
                lstm_r2,lstm_rmse,lstm_mae,]

In [ ]:
high_acc = []
good = []
mid = []
bad = []
for i in range(len(rnn_mape)):
    if lstm_mape[i]<=0.10:
        high_acc.append(lstm_mape[i])
    elif lstm_mape[i]<=0.20 and lstm_mape[i]>0.10:
        good.append(lstm_mape[i])
    elif lstm_mape[i]<=0.50 and lstm_mape[i]>0.20:
        mid.append(lstm_mape[i])
    elif lstm_mape[i]>0.50:
        bad.append(lstm_mape[i])

In [ ]:
print(len(rnn_mape))
print(len(lstm_mape))

In [ ]:
from statistics import mean 

print("Highly accuracte MAPE count: ", len(high_acc), "-> avg = ",mean(high_acc))
print("Good accuracte MAPE count: ", len(good),"-> avg = ",mean(good))
print("Reasonably good accuracte MAPE count: ", len(mid), "-> avg = ",mean(mid))
print("Inaccuracte MAPE count: ", len(bad))

In [ ]:
high_acc = []
good = []
mid = []
bad = []
for i in range(len(rnn_mape)):
    if rnn_mape[i]<=0.10:
        high_acc.append(rnn_mape[i])
    elif rnn_mape[i]<=0.20 and rnn_mape[i]>0.10:
        good.append(rnn_mape[i])
    elif rnn_mape[i]<=0.50 and rnn_mape[i]>0.20:
        mid.append(rnn_mape[i])
    elif rnn_mape[i]>0.50:
        bad.append(rnn_mape[i])

In [ ]:
from statistics import mean 

print("Highly accuracte MAPE count: ", len(high_acc), "-> avg = ",mean(high_acc))
print("Good accuracte MAPE count: ", len(good),"-> avg = ",mean(good))
print("Reasonably good accuracte MAPE count: ", len(mid), "-> avg = ",mean(mid))
print("Inaccuracte MAPE count: ", len(bad))

In [ ]:
fields = ['Name', 'RNN results', 'LSTM results']
filename = 'PriceVolBlock_Results_block10_w50.csv'
#RNN, LSTM Results look like: [[r2score, rsme, mae, accuracy1, f1_1, f1_indiv[0]_1,f1_indiv[1]_1, y_a.count(0), y_a.count(1)],..]
with open(filename, 'w') as csvfile:
    # creating a csv writer object
    csvwriter = csv.writer(csvfile)
 
    # writing the fields
    csvwriter.writerow(fields)
 
    # writing the data rows
    csvwriter.writerows(rows)

In [ ]:
print(len(rows))

In [ ]:
rnn_r2 = []
rnn_rmse = []
rnn_mae = []
rnn_mape = []

rnn_f1_1_e1 = []
rnn_f1_e1 = []
rnn_acc_e1 = []
rnn_per_e1 = []

rnn_f1_1_e2 = []
rnn_f1_e2 = []
rnn_acc_e2 = []
rnn_per_e2 = []

rnn_f1_1_e3 = []
rnn_f1_e3 = []
rnn_acc_e3 = []
rnn_per_e3 = []

rnn_f1_1_e4 = []
rnn_f1_e4 = []
rnn_acc_e4 = []
rnn_per_e4 = []

lstm_r2 = []
lstm_rmse = []
lstm_mae = []
lstm_mape = []

lstm_f1_1_e1 = []
lstm_f1_e1 = []
lstm_acc_e1 = []
lstm_per_e1 = []

lstm_f1_1_e2 = []
lstm_f1_e2 = []
lstm_acc_e2 = []
lstm_per_e2 = []

lstm_f1_1_e3 = []
lstm_f1_e3 = []
lstm_acc_e3 = []
lstm_per_e3 = []

lstm_f1_1_e4 = []
lstm_f1_e4 = []
lstm_acc_e4 = []
lstm_per_e4 = []

results_rows = []
for i in range (len(rows)):
    row = rows[i]
    rnn = row[1]
    rnn_r2.append(rnn[0])
    rnn_rmse.append(rnn[1])
    rnn_mae.append(rnn[2])
    rnn_mape.append(rnn[3])
    
    rnn_acc_e1.append(rnn[4][0])
    rnn_f1_e1.append(rnn[4][1])
    rnn_f1_1_e1.append(rnn[4][2])
    rnn_per_e1.append(rnn[4][3])

    rnn_acc_e2.append(rnn[5][0])
    rnn_f1_e2.append(rnn[5][1])
    rnn_f1_1_e2.append(rnn[5][2])
    rnn_per_e2.append(rnn[5][3])

    rnn_acc_e3.append(rnn[6][0])
    rnn_f1_e3.append(rnn[6][1])
    rnn_f1_1_e3.append(rnn[6][2])
    rnn_per_e3.append(rnn[6][3])

    rnn_acc_e4.append(rnn[7][0])
    rnn_f1_e4.append(rnn[7][1])
    rnn_f1_1_e4.append(rnn[7][2])
    rnn_per_e4.append(rnn[7][3])
    
    lstm = row[2]
    
    lstm_r2.append(lstm[0])
    lstm_rmse.append(lstm[1])
    lstm_mae.append(lstm[2])
    lstm_mape.append(lstm[3])
    
    lstm_acc_e1.append(lstm[4][0])
    lstm_f1_e1.append(lstm[4][1])
    lstm_f1_1_e1.append(lstm[4][2])
    lstm_per_e1.append(lstm[4][3])

    lstm_acc_e2.append(lstm[5][0])
    lstm_f1_e2.append(lstm[5][1])
    lstm_f1_1_e2.append(lstm[5][2])
    lstm_per_e2.append(lstm[5][3])

    lstm_acc_e3.append(lstm[6][0])
    lstm_f1_e3.append(lstm[6][1])
    lstm_f1_1_e3.append(lstm[6][2])
    lstm_per_e3.append(lstm[6][3])
    
    lstm_acc_e4.append(lstm[7][0])
    lstm_f1_e4.append(lstm[7][1])
    lstm_f1_1_e4.append(lstm[7][2])
    lstm_per_e4.append(lstm[7][3])
    
results_rows = [rnn_r2,rnn_rmse,rnn_mae,
                rnn_f1_1_e1,rnn_f1_e1,rnn_acc_e1,rnn_per_e1,
                rnn_f1_1_e2,rnn_f1_e2,rnn_acc_e2,rnn_per_e2,
                rnn_f1_1_e3,rnn_f1_e3,rnn_acc_e3,rnn_per_e3,
                rnn_f1_1_e4,rnn_f1_e4,rnn_acc_e4,rnn_per_e4,
                lstm_r2,lstm_rmse,lstm_mae,
                lstm_f1_1_e1,lstm_f1_e1,lstm_acc_e1,lstm_per_e1,
                lstm_f1_1_e2,lstm_f1_e2,lstm_acc_e2,lstm_per_e2,
                lstm_f1_1_e3,lstm_f1_e3,lstm_acc_e3,lstm_per_e3,
               lstm_f1_1_e4,lstm_f1_e4,lstm_acc_e4,lstm_per_e4]

In [ ]:
filename = 'ResultsRows1.csv'
with open(filename, 'w') as csvfile:
    # creating a csv writer object
    csvwriter = csv.writer(csvfile)
 
    # writing the data rows
    csvwriter.writerows(results_rows)

In [ ]:
high_acc = []
good = []
mid = []
bad = []
for i in range(len(rnn_mape)):
    if lstm_mape[i]<=0.10:
        high_acc.append(lstm_mape[i])
    elif lstm_mape[i]<=0.20 and lstm_mape[i]>0.10:
        good.append(lstm_mape[i])
    elif lstm_mape[i]<=0.50 and lstm_mape[i]>0.20:
        mid.append(lstm_mape[i])
    elif lstm_mape[i]>0.50:
        bad.append(lstm_mape[i])

In [ ]:
print("Highly accuracte MAPE count: ", len(high_acc), "-> avg = ",mean(high_acc))
print("Good accuracte MAPE count: ", len(good),"-> avg = ",mean(good))
print("Reasonably good accuracte MAPE count: ", len(mid), "-> avg = ",mean(mid))
print("Inaccuracte MAPE count: ", len(bad))

In [ ]:
for i in range(len(rnn_mape)):
    if lstm_mape[i]>0.50:
        print(i)

In [ ]:
high_acc = []
good = []
mid = []
bad = []
for i in range(len(rnn_mape)):
    if rnn_mape[i]<=0.10:
        high_acc.append(rnn_mape[i])
    elif rnn_mape[i]<=0.20 and rnn_mape[i]>0.10:
        good.append(lstm_mape[i])
    elif rnn_mape[i]<=0.50 and rnn_mape[i]>0.20:
        mid.append(rnn_mape[i])
    elif rnn_mape[i]>0.50:
        bad.append(rnn_mape[i])

In [ ]:
print("Highly accuracte MAPE count: ", len(high_acc), "-> avg = ",mean(high_acc))
print("Good accuracte MAPE count: ", len(good),"-> avg = ",mean(good))
print("Reasonably good accuracte MAPE count: ", len(mid), "-> avg = ",mean(mid))
print("Inaccuracte MAPE count: ", len(bad))

In [ ]:
ticker = ['HDFCBANK','MARUTI', 'M&M', 'ULTRACEMCO', 'GRASIM', 'PIDILITIND', 'LT', 'DLF','HINDUNILVR','ITC','RELIANCE','ONGC','UPL','ICICIBANK','SIEMENS',
          'ABB','TCS','INFY','COALINDIA','TATASTEEL','SUNPHARMA','DIVISLAB','ADANIPORTS','CONCOR','BHARTIARTL','IDEA']